In [1]:
import gdspy
import numpy as np
from element import *
from sketch import *
import imp

# x = PCB_CHIP(leftVia = True, rightVia = True)
# x.draw()

### Обозначения слоев
0 - металл с копланарами

1 - подложка

2 - металл с чипами

3 - vias

4 - порты

5 - технический уровень

6 - сверление на неполную глубину

# 16-pin держатель

In [2]:
imp.reload(gdspy)
from element import *
from sketch import *

sketch = Sketch('16-pin sample holder')
baseGeometry = gdspy.Round((0, 0), 53e3/2, tolerance=curve_tolerance)
for i in [0, 1, 2]:
    sketch.boolean(baseGeometry, 'or', i)

Rcpw = 44e3/2

R_center_position_hole = 53e3/2
D_positionHole = 2e3

a_screw = 10.5e3*2
D_screw = 3.2e3

portePerEdge = 4
portsNum = portePerEdge*4
singlePortAngle = 2*np.pi/portsNum

u_cpw = np.array([np.cos(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
                 -np.sin(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*Rcpw
h_position_hole = np.array([np.cos(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
                 -np.sin(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*R_center_position_hole
holeSize = 10.1e3

hole = ChipHole(a = holeSize, TH = 0.5e3, portsPerEdge = portePerEdge, portDisp = 1280*2,  throughAll = True)

for i in range(0, 2):
    if i%portePerEdge == 0:
        chipPort = PCB_CHIP(leftVia = True, rightVia = True)
    else:
        chipPort = PCB_CHIP(leftVia = True, rightVia = False)
    chipPort.mergeWithPort(hole.ports[i], 1)
    print(chipPort.position)
    
    smpPort = SMP_PCB()
    smpPort.move(u_cpw)
    smpPort.rotate(np.pi - (singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2))
    smpPort.rotate(singlePortAngle*i, center = (0,0))
    print(smpPort.position)
    
    if i%4 == 0 or i%4 == 3:
        cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 330, chessOrder = True)
    else:
        cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 550, chessOrder = True)
    
   
    sketch.addGeometry(cpw)
    sketch.addGeometry(smpPort)
    sketch.addGeometry(chipPort)

    print(i)
    
for i in range(0, portsNum):    
     if i%4 == 3 or i%4 == 1:
        positionHole = ThroughHole(D_positionHole)
        positionHole.move(h_position_hole)
        positionHole.rotate(singlePortAngle*i, center = (0,0))
        sketch.addGeometry(positionHole)
        
    
for i in [-1,1]:
    for j in [-1,1]:
        screwHole = ThroughHole(D_screw)
        screwHole.move((i*a_screw/2, j*a_screw/2))
        sketch.addGeometry(screwHole)
        
    
    
sketch.addGeometry(hole)
sketch.draw()
sketch.saveGDS('16pin low quality')
sketch.saveSVG('16pin low quality')

(6500.0, -3840.0)
(18292.331470655994, -12222.545126431249)
[array([ 8650., -3840.]), array([12468.48648263, -3840.        ]), array([13964.86297669, -9331.02312249]), array([ 15517.92539398, -10368.74625308])]
4950.0
0
(6500.0, -1280.0)
(21577.276168871067, -4291.987084354822)
[array([ 8650., -1280.]), array([11983.37442509, -1280.        ]), array([16602.0017154 , -3302.34346445]), array([18304.64106399, -3641.01948807])]
15741.0
1


# 8-pin держатель

In [3]:
# imp.reload(gdspy)
# from element import *
# from sketch import *

# sketch = Sketch('8-pin sample holder')
# baseGeometry = gdspy.Round((0, 0), 45e3/2)
# for i in [0, 1, 2]:
#     sketch.boolean(baseGeometry, 'or', i)


# Rsmp = 36e3/2
# angSMP = 30*np.pi/180/2
# angSMPdisp = -5*np.pi/180/2

# R_center_position_hole = 40e3/2
# D_positionHole = 2e3

# a_screw = 7e3*2
# D_screw = 3e3

# holeSize = 6.1e3

# hole = ChipHole(a = holeSize, TH = 0.5e3, portsPerEdge = 2, portDisp = 1280*2+300*2,  throughAll = True)
# sketch.addGeometry(hole)


# for j in range(0,4):
#     for i in range(0, 2):
#         if i == 0:
#             chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#             chipPort.mergeWithPort(hole.ports[j*2 + i], 1)
            
#             smpPort = SMP_PCB()
#             smpPort.move([Rsmp, 0])
#             smpPort.rotate(np.pi)
#             smpPort.rotate(-angSMP, center = (0, 0))
#             smpPort.rotate(-angSMPdisp)
#             smpPort.rotate(np.pi/2*j, center = (0, 0))
            
#             cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
#             sketch.addGeometry(cpw)
#             sketch.addGeometry(smpPort)
#             sketch.addGeometry(chipPort)

#         else:
#             chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#             chipPort.mergeWithPort(hole.ports[j*2 + i], 1)
            
#             smpPort = SMP_PCB()
#             smpPort.move([Rsmp, 0])
#             smpPort.rotate(np.pi)
#             smpPort.rotate(angSMP, center = (0, 0))
#             smpPort.rotate(angSMPdisp)
#             smpPort.rotate(np.pi/2*j, center = (0, 0))
   
#             cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
#             sketch.addGeometry(cpw)
#             sketch.addGeometry(smpPort)
#             sketch.addGeometry(chipPort)

#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move([R_center_position_hole, 0])
#     positionHole.rotate(np.pi/2*j, center = (0,0))
#     sketch.addGeometry(positionHole)
    
#     screwHole = ThroughHole(D_screw)
#     screwHole.move((a_screw/2, a_screw/2))
#     screwHole.rotate(np.pi/2*j, center = (0,0))
#     sketch.addGeometry(screwHole)
    
#     screwHole = ThroughHole(4e3)
#     screwHole.move((25e3/2, 25e3/2))
#     screwHole.rotate(np.pi/2*j, center = (0,0))
#     sketch.addGeometry(screwHole)



# sketch.draw()
# sketch.saveGDS('8pin')
# sketch.saveSVG('8pin')

# 8-pin держатель, 2 чипа интегрированы


In [4]:
# imp.reload(gdspy)
# from element import *
# from sketch import *

# sketch = Sketch('8-pin sample holder x2')

# disp = 13.2e3
# mainR = 43e3/2

# center1 = (-disp/2, 0)
# circ1 = gdspy.Round(center1, mainR)

# center2 = tuple(np.array(center1) + np.array([disp, 0]))
# circ2 = gdspy.Round(center2, mainR)

# baseGeometry = gdspy.boolean(circ1, circ2,'or')
# baseGeometry = gdspy.boolean(baseGeometry, gdspy.Rectangle([center1[0], -mainR],[center2[0], mainR]),'or')

# for i in [0, 1, 2]:
#     sketch.boolean(baseGeometry, 'or', i)


# Rsmp = 34e3/2
# angSMP = 30*np.pi/180/2
# angSMPdisp = -5*np.pi/180/2

# R_center_position_hole = 35e3/2
# D_positionHole = 2e3

# a_screw = 7e3*2
# D_screw = 3e3

# holeSize = 6e3

# hole1 = ChipHole(a = holeSize, TH = 0.5e3, portsPerEdge = 2, portDisp = 1280*2+300*2,  throughAll = True)
# hole1.move(center1)
# sketch.addGeometry(hole1)

# hole2 = ChipHole(a = holeSize, TH = 0.5e3, portsPerEdge = 2, portDisp = 1280*2+300*2,  throughAll = True)
# hole2.move(center2)
# sketch.addGeometry(hole2)

# for j in range(1, 4):
#     for i in range(0, 2):
#         chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#         chipPort.mergeWithPort(hole1.ports[j*2 + i], 1)
#         sketch.addGeometry(chipPort)
        
#         smpPort = SMP_PCB()
#         smpPort.move(center1)
#         smpPort.move([Rsmp, 0])
#         smpPort.rotate(np.pi)
        
#         if j == 2:
#             if i == 0:
#                 smpPort.rotate(-angSMP, center = center1)
#                 smpPort.rotate(-angSMPdisp)
#                 smpPort.rotate(np.pi/2*j, center = center1)
#             else:
#                 smpPort.rotate(angSMP, center = center1)
#                 smpPort.rotate(angSMPdisp)
#                 smpPort.rotate(np.pi/2*j, center = center1)
#             sketch.addGeometry(smpPort)

#         if j == 1:
#             if i == 0:
#                 smpPort.placeOn([chipPort.ports[0].position[0], Rsmp])
#                 smpPort.rotate(np.pi/2*j)
                
#             else:
#                 smpPort.rotate(25*np.pi/180, center = center1)
#                 smpPort.rotate(-angSMPdisp*2)
#                 smpPort.rotate(np.pi/2*j, center = center1)
#             sketch.addGeometry(smpPort)

#         if j == 3:
#             if i == 0:
#                 smpPort.rotate(-25*np.pi/180, center = center1)
#                 smpPort.rotate(angSMPdisp*2)
#                 smpPort.rotate(np.pi/2*j, center = center1)
#             else:
#                 smpPort.placeOn([chipPort.ports[0].position[0], -Rsmp])
#                 smpPort.rotate(np.pi/2*j)
#             sketch.addGeometry(smpPort)

#         cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
#         sketch.addGeometry(cpw)
            
# for j in [0, 1, 3]:
#     for i in range(0, 2):
#         chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#         chipPort.mergeWithPort(hole2.ports[j*2 + i], 1)
#         sketch.addGeometry(chipPort)
        
#         smpPort = SMP_PCB()
#         smpPort.move(center2)
#         smpPort.move([Rsmp, 0])
#         smpPort.rotate(np.pi)
        
#         if j == 0:
#             if i == 0:
#                 smpPort.rotate(-angSMP, center = center2)
#                 smpPort.rotate(-angSMPdisp)
#                 smpPort.rotate(np.pi/2*j, center = center2)
#             else:
#                 smpPort.rotate(angSMP, center = center2)
#                 smpPort.rotate(angSMPdisp)
#                 smpPort.rotate(np.pi/2*j, center = center2)
#             sketch.addGeometry(smpPort)

#         if j == 1:
#             if i == 0:
#                 smpPort.rotate(-25*np.pi/180, center = center2)
#                 smpPort.rotate(angSMPdisp*2)
#                 smpPort.rotate(np.pi/2*j, center = center2)
#             else:
#                 smpPort.placeOn([chipPort.ports[0].position[0], Rsmp])
#                 smpPort.rotate(np.pi/2*j)
#             sketch.addGeometry(smpPort)

#         if j == 3:
#             if i == 0:
#                 smpPort.placeOn([chipPort.ports[0].position[0], -Rsmp])
#                 smpPort.rotate(np.pi/2*j)
#             else:
#                 smpPort.rotate(25*np.pi/180, center = center2)
#                 smpPort.rotate(-angSMPdisp*2)
#                 smpPort.rotate(np.pi/2*j, center = center2)
#             sketch.addGeometry(smpPort)

#         cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
#         sketch.addGeometry(cpw)        

# for i in range(0, 2):
#         chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#         chipPort.mergeWithPort(hole1.ports[i], 1)
#         sketch.addGeometry(chipPort)
        
#         chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#         chipPort.mergeWithPort(hole2.ports[2*2 + i], 1)
#         sketch.addGeometry(chipPort)


# positionHole = ThroughHole(D_positionHole)
# positionHole.move([20e3, 12e3])
# sketch.addGeometry(positionHole)
# positionHole = ThroughHole(D_positionHole)
# positionHole.move([-20e3, 12e3])
# sketch.addGeometry(positionHole)
# positionHole = ThroughHole(D_positionHole)
# positionHole.move([20e3, -12e3])
# sketch.addGeometry(positionHole)
# positionHole = ThroughHole(D_positionHole)
# positionHole.move([-20e3, -12e3])
# sketch.addGeometry(positionHole)
    
# for j in range(0, 4):
#     if j == 1 or j == 3:
#         positionHole = ThroughHole(D_positionHole)
#         positionHole.move([R_center_position_hole, 0])
#         positionHole.rotate(np.pi/2*j, center = (0, 0))
#         sketch.addGeometry(positionHole)

        
#     if j == 1 or j == 2:
#         screwHole = ThroughHole(D_screw)
#         screwHole.move(center1)
#         screwHole.move((a_screw/2, a_screw/2))
#         screwHole.rotate(np.pi/2*j, center = center1)
#         sketch.addGeometry(screwHole)
    
#     if j == 3 or j == 0:
#         screwHole = ThroughHole(D_screw)
#         screwHole.move(center2)
#         screwHole.move((a_screw/2, a_screw/2))
#         screwHole.rotate(np.pi/2*j, center = center2)
#         sketch.addGeometry(screwHole)

        
# screwHole = ThroughHole(D_screw)
# screwHole.move((0, a_screw/2))
# sketch.addGeometry(screwHole)
# screwHole.move((0, -a_screw))
# sketch.addGeometry(screwHole)

# sketch.draw()
# sketch.saveGDS('8pin x2')
# sketch.saveSVG('8pin x2')

# 2-pin

In [5]:
# imp.reload(gdspy)
# from element import *
# from sketch import *

# sketch = Sketch('2-pin sample holder')

# a = 28e3
# b = 8e3

# baseGeometry = gdspy.Rectangle([-a/2, -b/2], [a/2, b/2])

# for i in [0, 1, 2]:
#     sketch.boolean(baseGeometry, 'or', i)
    

# holeSize = 4e3

# hole = ChipHole(a = holeSize, TH = 0.5e3, portsPerEdge = 1, portDisp = 0,  throughAll = True)
# sketch.addGeometry(hole)

# chipPort1 = PCB_CHIP(leftVia = True, rightVia = True)
# chipPort1.mergeWithPort(hole.ports[2], 1)
# sketch.addGeometry(chipPort1)

# chipPort2 = PCB_CHIP(leftVia = True, rightVia = True)
# chipPort2.mergeWithPort(hole.ports[0], 1)
# sketch.addGeometry(chipPort2)  

# smpPosition = 9.7e3

# smpPort1 = SMP_PCB()
# smpPort1.move([-smpPosition, 0])

# smpPort2 = SMP_PCB()
# smpPort2.move([smpPosition, 0])
# smpPort2.rotate(np.pi)

# sketch.addGeometry(smpPort1)  
# sketch.addGeometry(smpPort2)

# cpw1 = CPW(port1 = chipPort1.ports[0], port2 = smpPort1.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
# sketch.addGeometry(cpw1)  
# cpw2 = CPW(port1 = chipPort2.ports[0], port2 = smpPort2.ports[0], s = 1000, w = 330,  inter_dist = 500, chessOrder = False)
# sketch.addGeometry(cpw2)



# a_position = 9e3
# b_position = 8e3
# D_positionHole = 2e3

# for j in range(0,2):
#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move([a_position/2, b_position/2])
#     positionHole.rotate(np.pi*j, center = (0, 0))
#     sketch.addGeometry(positionHole)
    
#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move([a_position/2, -b_position/2])
#     positionHole.rotate(np.pi*j, center = (0, 0))
#     sketch.addGeometry(positionHole)

    
# viaDisp = 5.8e3
# viaLen = 5.8e3
# viaN = 8

# viaHole = Geometry()
# viaHole.boolean(gdspy.Round((0,0), 300/2), 'or', 3)
# viaHole.placeOn([-viaLen/2,viaDisp/2])
# sketch.addGeometry(viaHole)
# for i in range(0, viaN):
#     viaHole.move([viaLen/(viaN),0])
#     sketch.addGeometry(viaHole)

    
# viaHole.placeOn([-viaLen/2,-viaDisp/2])
# sketch.addGeometry(viaHole)
# for i in range(0, viaN):
#     viaHole.move([viaLen/(viaN),0])
#     sketch.addGeometry(viaHole)

# sketch.draw()
# sketch.saveGDS('2pin')
# sketch.saveSVG('2pin')

# Check PCB

In [6]:
# imp.reload(gdspy)
# from element import *
# from sketch import *

# sketch = Sketch('check pcb')

# a = 28e3
# b = 8e3

# baseGeometry = gdspy.Rectangle([-a/2, -b/2], [a/2, b/2])

# for i in [0, 1, 2]:
#     sketch.boolean(baseGeometry, 'or', i)


# smpPosition = 9.7e3

# smpPort1 = SMP_PCB()
# smpPort1.move([-smpPosition, 0])

# smpPort2 = SMP_PCB()
# smpPort2.move([smpPosition, 0])
# smpPort2.rotate(np.pi)

# sketch.addGeometry(smpPort1)  
# sketch.addGeometry(smpPort2)

# cpw = CPW(port1 = smpPort1.ports[0], port2 = smpPort2.ports[0], s = 1000, w = 330,  inter_dist = 300, chessOrder = False)
# sketch.addGeometry(cpw)  

# a_position = 9e3
# b_position = 8e3
# D_positionHole = 2e3

# for j in range(0,2):
#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move([a_position/2, b_position/2])
#     positionHole.rotate(np.pi*j, center = (0, 0))
#     sketch.addGeometry(positionHole)
    
#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move([a_position/2, -b_position/2])
#     positionHole.rotate(np.pi*j, center = (0, 0))
#     sketch.addGeometry(positionHole)

# sketch.draw()
# sketch.saveGDS('check PCB')
# sketch.saveSVG('check PCB')